<a href="https://colab.research.google.com/github/mykon23/AI-2020/blob/master/Activities/Activity15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##BM25 Function

In [1]:
!wget -O bm25.csv https://github.com/badriadhikari/AI-2020fall/blob/master/activities/bm25.csv?raw=True

--2020-10-18 02:27:18--  https://github.com/badriadhikari/AI-2020fall/blob/master/activities/bm25.csv?raw=True
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/badriadhikari/AI-2020fall/raw/master/activities/bm25.csv [following]
--2020-10-18 02:27:19--  https://github.com/badriadhikari/AI-2020fall/raw/master/activities/bm25.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/badriadhikari/AI-2020fall/master/activities/bm25.csv [following]
--2020-10-18 02:27:19--  https://raw.githubusercontent.com/badriadhikari/AI-2020fall/master/activities/bm25.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:

In [2]:
import pandas as pd
data = pd.read_csv('bm25.csv')
data.head()

,DocumentID,DocumentLength,FrequencyOfWord1,FrequencyOfWord2
0,1,7,1,1
1,2,17,2,4
2,3,8,0,0
3,4,25,5,0
4,5,11,2,3


Define BM25 Scores Function

In [3]:
import pandas as pd
import math

def get_bm25_scores( data, q_list, params ):

    ##Set the columns of interest
    doc_len = 'DocumentLength'
    doc_id = 'DocumentID'

    ##Get the average document length
    L = data[doc_len].sum(axis = 0)/len(data)

    k = params.get('k', 1.2)
    b = params.get('b', 0.75)

    ##Set the Adjusted query list to match the columns
    adj_q_list = [ 'FrequencyOf' + q for q in q_list ]

    ##Get the inverse document frequency for each word in the query
    idfs = get_idfs(data, adj_q_list)

    ##Use a DataFrame to support operating on all documents
    bm25 = pd.DataFrame()

    for q in adj_q_list:
        ##Acquire the bm25 score for the specified word across all documents
        bm_q = ( data[q] * (k + 1.0) )/( data[q] + k * (1.0 - b + b * data[doc_len]/L) )
        bm_q *= idfs[q]
        bm25[q] = bm_q

    ##Set the score as the sums of the individual word scores
    bm25['Score'] = bm25.sum(axis = 1)
    return pd.concat( [ data[doc_id], bm25['Score'] ], axis = 1)

##Function to get the Inverse Document Frequency
def get_idfs(data, q_list):
    idfs = {}
    for q in q_list:
        ##Set adjust df since df_q + 0.5 is used in two places
        df_adj = len( data[ data[q] != 0 ] ) + 0.5
        ##Add 1 to offset the -0.5 added from the adjusted df
        idfs[q] = math.log((len(data) - df_adj + 1.0)/(df_adj))
    return idfs


Get the Scores from Query and Display Results in Descending Order

In [4]:
##Set the query list and the parameters
q_list = [ 'Word1', 'Word2' ]
params = {'b': 0.75, 'k': 1.2 }

##Get the BM25 Scores of each Document
bm25_scores = get_bm25_scores(data, q_list, params)

In [5]:
##Display the scores in descending order
bm25_scores.sort_values(by=['Score'], ascending=False)

,DocumentID,Score
28,29,1.849590
4,5,1.794161
18,19,1.717198
1,2,1.704905
39,40,1.581831
0,1,1.477892
12,13,1.116345
15,16,1.042178
3,4,1.018938
14,15,0.965360
